In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from config import GameConfig
from constants import ACTION, BUY
from enums import StartingSplit
from env import DefaultEnvironment, FullEnvironment
from player import load_players
from simulations import simulate

In [3]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
# project_root = '/home/justiny/Princeton/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)
# logger.setLevel(ACTION)

## Rule-Based Bots ##

In [90]:
config = GameConfig(splits=[StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit], sandbox=True, num_players=2, must_include=[])
players = load_players(['BM', 'BM'], None, map_location='cpu', train=False)
agent = players[0]
env = FullEnvironment(config, players, logger=logger)

In [91]:
n = 10000

In [92]:
sim_stats = simulate(env, n, None, turn_log=True, action_log=True)

100%|██████████| 10000/10000 [03:08<00:00, 53.01it/s]
===SUMMARY===
{0: 5716, 1: 5574, 'ProvinceWins': 10000, 'Ties': 1290}


In [95]:
data_path = os.path.join(project_root, 'data', 'bm-bm-10k-stats')

In [96]:
save(data_path, sim_stats)

## Tree Simulations ##

In [6]:
from mcts import GameTree

In [7]:
tree_name = 'full-tree-paper-30k-boost'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog-full-paper'
rollout_path = os.path.join(model_dir, rollout_name)

In [9]:
tree_name = 'r-r-mcts-bm-warm-start-10k-srat-sandbox-ucb1'
rollout_name = 'default-sandbox-mlog-10k-v2'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [8]:
tree = GameTree.load(tree_path, train=False, _skip_level=False, selection='max')

In [72]:
tree.selection = 'max'
tree.selection

'max'

In [13]:
tree.skip_level = True

In [42]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
splits = [StartingSplit.Starting34Split, StartingSplit.Starting34Split]
# splits = [StartingSplit.Starting25Split, StartingSplit.Starting25Split]
# config = GameConfig(splits=splits, sandbox=True, num_players=2, must_include=[])
config = GameConfig(splits=splits, sandbox=False, must_include=default_kingdom)
players = load_players(['DW', 'UCT'], [rollout_path], tree=tree, rollout_type='mlog', use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)[0]
# players = [agent, agent]

env = DefaultEnvironment(config, players, logger=logger)

In [10]:
logger.setLevel(logging.ERROR)

In [43]:
sim_data = simulate(env, 1000, tree, turn_log=True, action_log=False, card_log=True)

100%|██████████| 1000/1000 [00:49<00:00, 20.06it/s]
===SUMMARY===
{0: 657, 1: 355, 'ProvinceWins': 992, 'Ties': 12}


In [34]:
logger.setLevel(BUY)

In [43]:
sim_data = simulate(env, 1, tree, turn_log=False, action_log=False)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:Player 0, Turn: 0: None
BUY:Selection: 2412
BUY:Player 1, Turn: 0: Witch
BUY:Player 0, Turn: 1: Witch
BUY:Selection: 22
BUY:Player 1, Turn: 1: Moat
BUY:Player 0, Turn: 2: Silver
BUY:Player 1, Turn: 2: Silver
BUY:Player 0, Turn: 3: Witch
BUY:Rollout
BUY:Player 1, Turn: 3: Silver
BUY:Player 0, Turn: 4: Silver
BUY:Rollout
BUY:Player 1, Turn: 4: Silver
BUY:Player 0, Turn: 5: Silver
BUY:Rollout
BUY:Player 1, Turn: 5: Gold
BUY:Player 0, Turn: 6: Silver
BUY:Rollout
BUY:Player 1, Turn: 6: None
BUY:Player 0, Turn: 7: Silver
BUY:Rollout
BUY:Player 1, Turn: 7: Silver
BUY:Player 0, Turn: 8: Silver
BUY:Rollout
BUY:Player 1, Turn: 8: Province
BUY:Player 0, Turn: 9: Gold
BUY:Rollout
BUY:Player 1, Turn: 9: Silver
BUY:Player 0, Turn: 10: Silver
BUY:Rollout
BUY:Player 1, Turn: 10: Gold
BUY:Player 0, Turn: 11: Gold
BUY:Rollout
BUY:Player 1, Turn: 11: Gold
BUY:Player 0, Turn: 12: Silver
BUY:Rollout
BUY:Player 1, Turn: 12: Silver
BUY:Player 0, Turn: 13: Province
BUY

In [45]:
data_dir = os.path.join(project_root, 'data')
data_name ='dw34-uct34-1k-data'
data_path = os.path.join(data_dir, data_name)

In [46]:
save(data_path, sim_data)